In [1]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector
from datetime import date

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.17.2'

In [3]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [5]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'sec-ch-ua': 'Google Chrome;v="88", "Chromium";v="88", ";Not A Brand";v="99"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': 'macOS', 'sec-fetch-site': 'none', 'sec-fetch-mod': '', 'sec-fetch-user': '?1', 'accept-encoding': 'gzip, deflate', 'accept-language': 'en-US,en;q=0.9,de;q=0.5'}


In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15


In [7]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [8]:
# Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
option = webdriver.ChromeOptions()

# Going undercover:
option.add_argument("--incognito")


# # Consider this if the application works and you know how it works for speed ups and rendering!

option.add_argument('--headless=chrome')
user_agent = user_agent_random
option.add_argument(f"user-agent={user_agent}")

# Thêm header vào Options
headers = header_browser_random

for key, value in headers.items():
    option.add_argument(f"--header={key}:{value}")

In [9]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)

driver.get("https://itviec.com/it-jobs?")

job_count = driver.find_element(By.CLASS_NAME,'headline-total-jobs').text
number_of_jobs = job_count.split()[0]
if (int(number_of_jobs) % 20 == 0):
    max_page  = int(number_of_jobs) / 20 
else:
    max_page = math.floor(int(number_of_jobs) / 20) + 1
print(max_page)
driver.quit()
job_urls = [] #Mảng lưu trữ tất cả url lấy được từ web

31


In [10]:
page_url ="https://itviec.com/it-jobs?page={}"
for page_number in range(1, int(max_page + 1)):
# for page_number in range(1, 3):
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]
    user_agent = user_agent_random

    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
        
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
    driver.get(page_url.format(page_number))
    sleep(randint(3, 5))
    jobs = driver.find_elements(By.CLASS_NAME,"job-card")
    for job in jobs:
        job_url = job.find_element(By.CSS_SELECTOR,'a').get_attribute("href")
        job_urls.append(job_url)
    # sleep(randint(6, 10))
    driver.quit()

In [11]:
conn = mysql.connector.connect(
    host='103.56.158.31',
    port = 3306,
    user= 'tuyendungUser',
    password='sinhvienBK',
    database= 'ThongTinTuyenDung'
)
cursor = conn.cursor()

In [12]:
sql = 'INSERT IGNORE INTO Stg_ThongTin_raw(Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong) VALUES (%s, %s, %s, %s,%s,%s,%s, %s, %s, %s, %s, %s, %s, %s,%s)'

In [13]:
sql_link = 'SELECT Link FROM Stg_ThongTin_raw where Web =\'ITViec\''

In [14]:
cursor.execute(sql_link)
result = cursor.fetchall()
remove_url_list = [row[0] for row in result]
print("Số url cào về: ", len(job_urls))
for job_url in remove_url_list:
    if job_url in job_urls:
        job_urls.remove(job_url)
print("Đã lấy link thành công")
print("Số lượng link mới sau xử lí lấy được: ", len(job_urls))

Số url cào về:  617
Đã lấy link thành công
Số lượng link mới sau xử lí lấy được:  375


In [15]:
if len(job_urls) > 0:
    for i in range(len(job_urls)):
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
        driver.get(job_urls[i])
        print("Đang xử lí....", job_urls[i])
        sleep(randint(1, 3))
        Web = 'ITViec'
        Nganh = 'IT'
        Link = job_urls[i]
        TenCV = driver.find_element(By.CLASS_NAME, 'ipt-md-6').text
        CongTy = driver.find_element(By.CLASS_NAME, 'employer-name').text
        TinhThanh = driver.find_elements(By.CSS_SELECTOR, '[class="normal-text text-rich-grey"]')[0].text
        Luong = 'Không có'
        LoaiHinh = driver.find_element(By.CSS_SELECTOR, '[class="normal-text text-rich-grey ms-1"]').text
        KinhNghiem = "Không có"
        CapBac = 'Không có'
        HanNopCV = date.today()
        try:
            YeuCau = driver.find_elements(By.CLASS_NAME, 'imy-5')[1].text
        except:
            YeuCau = ""
        try:
            MoTa = driver.find_elements(By.CLASS_NAME, 'imy-5')[0].text
        except:
            MoTa = ""
        try:
            PhucLoi = driver.find_elements(By.CLASS_NAME, 'imy-5')[2].text
        except:
            PhucLoi = ""
        SoLuong = '1'
        cursor.execute(sql, (Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong))
        conn.commit()
        driver.quit()
    cursor.close()
    
    
    
    
    
    conn.close()
    print("Đã thêm tin thành công")
else:
    print("Không có tin mới để thêm")

Đang xử lí.... https://itviec.com/it-jobs/embedded-software-developer-integration-automotive-luxoft-vietnam-4216?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/c-embedded-engineers-regular-senior-automotive-luxoft-vietnam-2508?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/it-business-analyst-english-fintech-kbtg-vietnam-5549?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/ui-ux-designer-athenapros-5059?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/ky-su-quan-tri-mang-cloud-sunteco-2729?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/react-native-mobile-developer-ios-android-thai-son-soft-3733?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/database-administrator-13t-luong-di-lam-sau-tet-viettel-post-a-member-of-viettel-group-1524?lab_feature=preview_jd_page
Đang xử lí.... https://itviec.com/it-jobs/senior-c-engineer-windows-app-naver-vietnam-3911?lab_